<a href="https://colab.research.google.com/github/DURUII/HIMIA-course/blob/main/DURUII/%E3%80%90A0%E3%80%91nn.Conv1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn as nn
import torch

# 卷积

博客网址：http://t.csdn.cn/TGS1Y

时延神经网络（[TDNNs](https://www.inf.ufrgs.br/~engel/data/media/file/cmp121/waibel89_TDNN.pdf)）常用于声纹识别领域，例如著名的X-VECTOR基础结构就是TDNNs；它可以视作一维卷积神经网络（1-d CNNs），甚至有人认为TDNNs是CNNs/LeNet-5的早期版本。

计算机视觉盛行的如今，数字图像处理中**二维卷积**（空域滤波/模版运算/互相关运算）对于我们而言也不再陌生，无非就是中心数值等于按元素相乘后再相加嘛。

可是，到底如何理解**一维卷积**？它和二维卷积又有什么联系？

不妨通过实验，一探究竟。

## 单输入/单输出通道

首先，我们暂时忽略通道。假定输入为$[1, 2, -1, 1, -3]$，核函数为$[1, 0, -1]$**（卷积核是一维的，移动方向也自然是一维的）**，如果卷积核在输入向量上移动，请问结果是多少呢？可以看看我有没有算错。

![](https://img-blog.csdnimg.cn/1da61a49f0344438b352ac7a0ba9a71c.png)

顺带一提，这里输入是一行五列的矩阵。不过换一个角度，我们也可以认为每次输入的是3帧信息，并按照某种权重加和汇入1个新帧。

这里的3是卷积核尺寸($kernel\ size$)，从共享参数的全连接输入的角度说，3也是上下文所涵盖的范围,$(n-1,n+1) \ or \ \{n-1, n, n+1\}$。

In [2]:
# 输入
input = torch.tensor(
    [
        [
            [1, 2, -1, 1, -3],
        ]
    ],
    dtype=torch.float,
)

conv1d = nn.Conv1d(
    in_channels=1,
    out_channels=1,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    bias=False,
    padding_mode="zeros",
)


# 卷积核
conv1d.weight = nn.Parameter(
    torch.tensor(
        [
            [
                [1.0, 0, -1],
            ]
        ]
    )
)

# 输出
conv1d(input)

tensor([[[2., 1., 2.]]], grad_fn=<ConvolutionBackward0>)

## 多输入/单输出通道

但是，一般说来，TDNNs输入的声学特征并不是一个向量，而是一个矩阵，横轴（宽度）与时间有关【不固定】，纵轴（高度）和频率相关【**固定**】，所以它有时也被称为时频谱图（Spectrogram）。

![](https://img-blog.csdnimg.cn/img_convert/f8a419a0a05853b4042c85ac7f1daf78.png)

图片来自[MathWorks]( )。


当然，上文所指的广义的时频谱也分很多种（常见的有MFCC、MFBank、MelSpec等），对应各种不同的处理方式（例如，模拟人耳对频率、声强的非线性，平稳信号等），不过它们一般都要经历短时傅里叶变换（分帧、加窗、离散傅里叶变换）。

你还可以在 *[CHROME MUSIC LAB:SPECTROGRAM](https://musiclab.chromeexperiments.com/spectrogram/)*  上，亲自动手玩玩可视化自己的或自然的声音。

![截图](https://img-blog.csdnimg.cn/12aa55c38418420ab5aca5c8a0a69a80.png)



我们暂时忽略输出通道。从一维卷积的视角说，[频率，时间] 应当被视作 [通道数，输入长度]，即每个通道各有一个卷积核或一组共享权重，每个通道独立计算再把各个通道的结果相加。

假定输入为双通道（例如表示高频和低频）$$[[1, 2, -1, 1, -3],[3, 1, 0, -1, 2]]$$各通道上的核函数分别为$[1, 0, -1]$和$[0.5, 0, 0.5]$，请问这次结果又是多少呢？相信你肯定比我算的快。

![](https://img-blog.csdnimg.cn/513fb240e9e84f86b6421758fc0d8efb.png)

In [3]:
# 输入
input = torch.tensor([[[1, 2, -1, 1, -3], [3, 1, 0, -1, 2]]], dtype=torch.float)

conv1d = nn.Conv1d(
    in_channels=2,
    out_channels=1,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    bias=False,
    padding_mode="zeros",
)

# 卷积核
conv1d.weight = nn.Parameter(
    torch.tensor(
        [
            [[1.0, 0, -1], [0.5, 0, 0.5]],
        ]
    )
)

# 输出
conv1d(input)

tensor([[[3.5000, 1.0000, 3.0000]]], grad_fn=<ConvolutionBackward0>)

## 多输入/多输出通道

最后，该如何理解输出通道呢？其实无非就是把单输出通道的那一份卷积核，扩增几倍而已。例如，仍然假定输入为双通道$$[[1, 2, -1, 1, -3],[3, 1, 0, -1, 2]]$$不过此时我们考虑简化的双**输出**通道的情况：

第一份输出：各输入通道上的核函数分别为$[1, 0, -1]$和$[0.5, 0, 0.5]$；

第二份输出：各输入通道上的核函数仍然是$[1, 0, -1]$和$[0.5, 0, 0.5]$。

那么，我们就可以得到双倍的输出。

In [4]:
# 输入
input = torch.tensor([[[1, 2, -1, 1, -3], [3, 1, 0, -1, 2]]], dtype=torch.float)

conv1d = nn.Conv1d(
    in_channels=2,
    out_channels=2,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    bias=False,
    padding_mode="zeros",
)

# 卷积核
conv1d.weight = nn.Parameter(
    torch.tensor(
        [
            [[1.0, 0, -1], [0.5, 0, 0.5]],
            [[1.0, 0, -1], [0.5, 0, 0.5]],
        ]
    )
)

# 输出
conv1d(input)

tensor([[[3.5000, 1.0000, 3.0000],
         [3.5000, 1.0000, 3.0000]]], grad_fn=<ConvolutionBackward0>)

## 一维卷积/二维卷积

其实，如果将各个通道的一维卷积核拼接起来：所谓的一维卷积在各通道上的**加和**，完全可以用二维卷积核$[channels, kernel\_ size]$按元素相乘后再**相加**理解。

![](https://github.com/DURUII/HIMIA-course/blob/main/DURUII/res/conv1d_2d.svg?raw=1)

所以，许多博主认为**一维卷积不代表卷积核是一维的，只代表卷积核的移动方向是一维的**；这种说法，从某种角度说，也有一定的道理。
![请添加图片描述](https://img-blog.csdnimg.cn/3bbb29e5a71344e885000b98032fc7e0.png)


不过值得注意的是，Pytorch中Conv1d和Conv2d输入维度不一样。

例如，
一维卷积 $[batch, channels, length]$，
二维卷积 $[batch, channels, height, width]$。

In [5]:
# 输入
input = torch.tensor([[[1, 2, -1, 1, -3], [3, 1, 0, -1, 2]]], dtype=torch.float)

input = input.unsqueeze(dim=1)

conv2d = nn.Conv2d(
    in_channels=1,
    out_channels=2,
    kernel_size=(input.shape[2], 3),
    stride=1,
    padding=0,
    dilation=1,
    bias=False,
    padding_mode="zeros",
)

# 卷积核
conv2d.weight = nn.Parameter(
    torch.tensor(
        [
            [[[1.0, 0, -1], [0.5, 0, 0.5]]],
            [[[1.0, 0, -1], [0.5, 0, 0.5]]],
        ]
    )
)

# 输出
conv2d(input)

tensor([[[[3.5000, 1.0000, 3.0000]],

         [[3.5000, 1.0000, 3.0000]]]], grad_fn=<ConvolutionBackward0>)

![](https://github.com/DURUII/HIMIA-course/blob/main/DURUII/res/v1_or_v2.png?raw=1)

# 参考资料

1. Daniel Povey et al. “Semi-Orthogonal Low-Rank Matrix Factorization for Deep Neural Networks..” conference of the international speech communication association(2018): n. pag.

2. Zhang, A. , et al. "Dive into Deep Learning." (2021).

3. https://ww2.mathworks.cn/help/audio/ug/cocktail-party-source-separation-using-deep-learning-networks.html